In [ ]:
!pip install google-api-python-client

In [ ]:
!pip install google-auth
!pip install google-auth-oauthlib

In [ ]:
!pip install -qU langchain-google-vertexai



In [ ]:
!pip install langchain-core

In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Path to your service account key file
SERVICE_ACCOUNT_FILE = "prj-uc-genai-labs-8859d31dca67.json"

# Define the scopes
SCOPES = ["https://www.googleapis.com/auth/cloud-platform"]

# Authenticate and construct service
credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

In [ ]:
!pip install vertexai

In [ ]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part

def generate_video_description(project_id, location, model_name, video_file_uri, prompt):
    vertexai.init(project=project_id, location=location)

    model = GenerativeModel(model_name)

    video_file = Part.from_uri(video_file_uri, mime_type="video/mp4")

    contents = [video_file, prompt]

    response = model.generate_content(contents)
    return response.text

# Example usage
project_id = "prj-uc-genai-labs"
location = "us-central1"
model_name = "gemini-1.5-pro-preview-0409"
video_file_uri = "gs://workshop_pablo_24/0424.mp4"
prompt = """
  Provide a description of the video.
  The description should also contain anything important which people say in the video.
"""

description = generate_video_description(project_id, location, model_name, video_file_uri, prompt)
print(description)

Create Trello account
Crearte power up
Create api key

In [ ]:
def create_trello_card(name, due, start):
    import requests
    import json

    url = "https://api.trello.com/1/cards"

    headers = {"Accept": "application/json"}

    query = {
        "idList": "662a0837e895ddb3063a6277",
        "key": "28c3d510ab4b942cf31f2993d4059942",
        "token": "ATTAf9894d9fcfa1a44dd9653a65f141f58e7503c196515909dd2eea97fe41b3ed43D5A5A316",
        "name": name,
        "due": due,
        "start": start,
    }

    response = requests.request("POST", url, headers=headers, params=query)

    print(
        json.dumps(
            json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": ")
        )
    )


Create pydantic model

In [ ]:
from pydantic import BaseModel, Field
from typing import Optional
from datetime import datetime

class CreateTrelloCard(BaseModel):
    """
    Function that calls the Trello API to create a new card.
    
    """
    name: str = Field(..., description="Name of the card")
    due: Optional[datetime] = Field(...,description="Name of the card")
    start: Optional[datetime] = Field(...,description="Name of the card")

Bind tools to Langchain

In [21]:
from langchain_google_vertexai import ChatVertexAI

In [ ]:
tools = [CreateTrelloCard]
llm = ChatVertexAI(
    model="gemini-1.5-pro-preview-0409",
    credentials=credentials,
    project_id="prj-uc-genai-labs",
    location="us-central1",
)


llm_with_tools = llm.bind_tools(tools)